In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the necessary libraries

In [ ]:
# Load libraries
import pandas as pd
from sklearn import preprocessing
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow
tensorflow.random.set_seed(0)
from numpy.random import seed
seed(1)

## Load and get a glimpse of the training and test sets

In [ ]:
# Load training data
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
len(train_data)

In [ ]:
train_data.head()

In [ ]:
# Load testing data
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
len(test_data)

In [ ]:
test_data.head()

Check for missing values in the training data:

In [ ]:
# Get the number of missing data points per column
missing_values_count_train = train_data.isnull().sum()
print(missing_values_count_train)

Check for missing values in the test data:

In [ ]:
# Get the number of missing data points per column
missing_values_count_test = test_data.isnull().sum()
print(missing_values_count_test)

Check the list of features available:

In [ ]:
print(train_data.columns)

Obtain the number of classes, and values in each:

In [ ]:
num_of_classes = train_data['target'].value_counts()
print(num_of_classes)

## Pre-processing of data

In [ ]:
le = LabelEncoder()
train_data['target'] = le.fit_transform(train_data['target'])

In [ ]:
train_corr = train_data.corr()
train_corr.head()

In [ ]:
features_list = list(train_data.columns)
features_num = features_list[1:-1]
#features_num = ['feature_42', 'feature_17', 'feature_11', 'feature_36', 'feature_13', 'feature_30', 'feature_39', 'feature_52', 'feature_1', 'feature_64', 'feature_15', 'feature_2', 'feature_27', 'feature_53', 'feature_19', 'feature_20', 'feature_71', 'feature_38', 'feature_55', 'feature_24', 'feature_43', 'feature_61', 'feature_4', 'feature_0', 'feature_22', 'feature_18', 'feature_49', 'feature_47', 'feature_12', 'feature_29', 'feature_66', 'feature_7', 'feature_40', 'feature_54', 'feature_69', 'feature_60', 'feature_68', 'feature_48', 'feature_23', 'feature_37', 'feature_67', 'feature_6', 'feature_8', 'id', 'feature_33', 'feature_62', 'feature_65', 'feature_63', 'feature_31', 'feature_44']
preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
)

y = train_data['target']
X = preprocessor.fit_transform(train_data[features_num])

test_X = preprocessor.transform(test_data[features_num])

Visualizing the class statistics below:

In [ ]:
# Check summary of each class
train_data.groupby('target').describe()

In [ ]:
print(X.shape)

## XGB Classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error

y = train_data['target']
lc = LabelEncoder() 
lc = lc.fit(y) 
lc_y = lc.transform(y)

features_list = list(train_data.columns)
features_num = features_list[1:-1]
#features_num = ['feature_42', 'feature_17', 'feature_11', 'feature_36', 'feature_13', 'feature_30', 'feature_39', 'feature_52', 'feature_1', 'feature_64', 'feature_15', 'feature_2', 'feature_27', 'feature_53', 'feature_19', 'feature_20', 'feature_71', 'feature_38', 'feature_55', 'feature_24', 'feature_43', 'feature_61', 'feature_4', 'feature_0', 'feature_22', 'feature_18', 'feature_49', 'feature_47', 'feature_12', 'feature_29', 'feature_66', 'feature_7', 'feature_40', 'feature_54', 'feature_69', 'feature_60', 'feature_68', 'feature_48', 'feature_23', 'feature_37', 'feature_67', 'feature_6', 'feature_8', 'id', 'feature_33', 'feature_62', 'feature_65', 'feature_63', 'feature_31', 'feature_44']
preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
)

X = preprocessor.fit_transform(train_data[features_num])
test_X = preprocessor.transform(test_data[features_num])

# Define the model
xgb_model = XGBClassifier(n_estimators = 50)

# Fit the model
xgb_model.fit(X,lc_y)

## Obtaining the test predictions

In [ ]:
test_preds = xgb_model.predict_proba(test_X)
print(test_preds[0])

## Preparing the format for submission

In [ ]:
# The lines below shows how to save predictions in format used for competition scoring.
output = pd.DataFrame({'id': test_data.id,
                       'Class_1': test_preds[:, 0],
                       'Class_2': test_preds[:, 1],
                       'Class_3': test_preds[:, 2],
                       'Class_4': test_preds[:, 3],
                       'Class_5': test_preds[:, 4],
                       'Class_6': test_preds[:, 5],
                       'Class_7': test_preds[:, 6],
                       'Class_8': test_preds[:, 7],
                       'Class_9': test_preds[:, 8],
                      })

output.to_csv('submission_tps.csv', index=False)

Visualizing the submission file below:

In [ ]:
output.head()

In [ ]:
from IPython.display import FileLink
FileLink('./submission_tps.csv')